In [58]:
import numpy as np
import pandas as pd
import requests

In [216]:
# Define player of interest
name = 'Jimmy Butler'
# Get player ID number
player_id = get_player(name)
# Define seasons of interest
seasons = list(np.arange(2018, 2023, 1, dtype=int))
# Create query string for the seasons of interest
season_query = ['&seasons[]={0}'.format(season) for season in seasons]
# Initialize list for DataFrames for future concatenation
data = []
# Iterate over pages - THIS METHOD IS SHIT, FIX IT
for i in range(0, 50):
    # Define request URL
    url = 'https://www.balldontlie.io/api/v1/stats?' + 'page={0}'.format(i) + ''.join(season_query) + '&player_ids[]={0}'.format(79)
    # Get player data into DataFrame
    temp = pd.DataFrame.from_dict(requests.get(url).json()['data'])
    # Unravel game data
    try:
        temp = pd.concat([temp, pd.DataFrame(list(temp['game'].values))], axis=1)
        # Append DataFrame to list
        data.append(temp)
    except:
        break
# Concatenate
data = pd.concat(data).sort_values('date').drop_duplicates('date').reset_index()

In [194]:
def get_player(name):
    
    ''' 
    Get player ID for identification purposes during stats retrieval. 
    
    Input: name (str)
    Output: id_ (int)
    '''
    
    # Get first and last name
    f, l = name.split(' ')
    # Define URL for player search
    url = 'https://www.balldontlie.io/api/v1/players?search={0}+{1}'.format(f.lower(), l.lower())
    # Try to find the player, return None if player not found
    try:
        id_ = requests.get(url).json()['data'][0]['id']
    except:
        print('No player found for this name.')
        id_ = None
        
    return id_